<a href="https://colab.research.google.com/github/paulovitornovaes/Algoritmos-e-Estruturas-de-Dados/blob/main/TrabalhoFinalBD3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Big Data: Como instalar o PySpark no Google Colab
Atualizado para a versão 3.4.0

Como instalar o PySpark no Google Colab é uma dúvida comum entre aqueles que estão migrando seus projetos de Data Science para ambientes na nuvem.

O termo Big Data está cada vez mais presente, e mesmo projetos pessoais podem assumir uma grande dimensionalidade devido à quantidade de dados disponíveis.

Para analisar grandes volumes de dados, Big Data, com velocidade, o Apache Spark é uma ferramenta muito utilizada, dada a sua capacidade de processamento de dados e computação paralela.

O Spark foi pensado para ser acessível, oferecendo diversas APIs e frameworks em Python, Scala, SQL e diversas outras linguagens.

## PySpark no Google Colab

PySpark é a interface alto nível que permite você conseguir acessar e usar o Spark por meio da linguagem Python. Usando o PySpark, você consegue escrever todo o seu código usando apenas o nosso estilo Python de escrever código.

Já o Google Colab é uma ferramenta incrível, poderosa e gratuita – com suporte de GPU inclusive. Uma vez que roda 100% na nuvem, você não tem a necessidade de instalar qualquer coisa na sua própria máquina.

No entanto, apesar da maioria das bibliotecas de Data Science estarem previamente instaladas no Colab, o mesmo não acontece com o PySpark. Para conseguir usar o PySpark é necessário alguns passos intermediários, que não são triviais para aqueles que estão começando.

Dessa forma, este é um tutorial simples e direto ensinando a instalar as dependências e a biblioteca.

## Instalando o PySpark no Google Colab

Instalar o PySpark não é um processo direto como de praxe em Python. Não basta usar um pip install apenas. Na verdade, antes de tudo é necessário instalar dependências como o Java 8, Apache Spark 3.4 junto com o Hadoop 3.3.

In [ ]:
#Instalar as dependências

#Instalar Java 8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

#Realizar o download do Spark
!wget -q https://dlcdn.apache.org/spark/spark-3.4.0/spark-3.4.0-bin-hadoop3.tgz

#Descompartar o arquivo baixado
!tar xf spark-3.4.0-bin-hadoop3.tgz

#Instalando a findspark
!pip install -q findspark

A próxima etapa é configurar as variáveis de ambiente, pois isso habilita o ambiente do Colab a identificar corretamente onde as dependências estão rodando.

Para conseguir “manipular” o terminal e interagir como ele, você pode usar a biblioteca os.

In [ ]:
#Configurar as variáveis de ambiente

#Importando a biblioteca os
import os

#Definindo a variável de ambiente do Java
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

#Definindo a variável de ambiente do Spark
os.environ["SPARK_HOME"] = "/content/spark-3.4.0-bin-hadoop3"

#Importando a findspark
import findspark

#Iniciando o findspark
findspark.init('spark-3.4.0-bin-hadoop3')

Com tudo pronto, vamos rodar uma sessão local para testar se a instalação funcionou corretamente.

In [ ]:
# iniciar uma sessão local
from pyspark.sql import SparkSession
import time
sc = SparkSession.builder.master('local[*]').config('spark.ui.port', '4050').getOrCreate()

In [ ]:
#Executando apenas a variável sc em uma célula veremos algumas informações sobre a sessão Spark criada.
sc

Importado dados de exemplo

In [ ]:
#Importando os dados do Airbnb

#Download do http para arquivo local
!wget --quiet --show-progress http://data.insideairbnb.com/brazil/rj/rio-de-janeiro/2023-03-28/visualisations/listings.csv

#O arquivo está disponível no diretório “/content/”. Agora, vamos levar esse arquivo arquivo *.csv para dentro do Spark:

listings.csv        100%[===================>]   4.37M  6.30MB/s    in 0.7s    


In [ ]:
#Carregar dados do Airbnb
df_spark = sc.read.option("delimiter", ",").csv("./listings.csv", inferSchema=True, header=True)

In [ ]:
#Verificando o tipo de objeto criado
type(df_spark)

pyspark.sql.dataframe.DataFrame

Perceba que é um formato específico de DataFrame, diferente do conhecido pandas.DataFrame, portanto não é possível utilizar os métodos do Pandas diretamente sobre esse objeto. Mas fique tranquilo que o PySpark possui uma enorme quantidade de métodos para serem aplicados aos Spark Dataframes.

In [ ]:
# Espiando o dataset
df_spark.show(truncate=False)

+------+--------------------------------------------------+-------+------------------------+-------------------+---------------+----------+----------+---------------+-----+--------------+-----------------+-----------+-----------------+------------------------------+----------------+---------------------+-------+
|id    |name                                              |host_id|host_name               |neighbourhood_group|neighbourhood  |latitude  |longitude |room_type      |price|minimum_nights|number_of_reviews|last_review|reviews_per_month|calculated_host_listings_count|availability_365|number_of_reviews_ltm|license|
+------+--------------------------------------------------+-------+------------------------+-------------------+---------------+----------+----------+---------------+-----+--------------+-----------------+-----------+-----------------+------------------------------+----------------+---------------------+-------+
|17878 |Very Nice 2Br in Copacabana w. balcony, fast WiFi 

In [ ]:
# Verificando o schema() deste sparkdataframe
df_spark.printSchema()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- host_id: string (nullable = true)
 |-- host_name: string (nullable = true)
 |-- neighbourhood_group: string (nullable = true)
 |-- neighbourhood: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- room_type: string (nullable = true)
 |-- price: string (nullable = true)
 |-- minimum_nights: string (nullable = true)
 |-- number_of_reviews: string (nullable = true)
 |-- last_review: string (nullable = true)
 |-- reviews_per_month: string (nullable = true)
 |-- calculated_host_listings_count: string (nullable = true)
 |-- availability_365: string (nullable = true)
 |-- number_of_reviews_ltm: double (nullable = true)
 |-- license: integer (nullable = true)



Perceba que a variável number_of_reviews, apesar de ser numérica e contínua, foi lida como string. Portanto, precisamos mudar o tipo da variável de string para int conforme comandos a seguir:

In [ ]:
#Transformando o atributo number_of_reviews em numerica
df_spark = df_spark.withColumn("number_of_reviews",df_spark.number_of_reviews.cast('int'))

#Verificando a transformação
df_spark.select('number_of_reviews').printSchema()

root
 |-- number_of_reviews: integer (nullable = true)



In [ ]:
#Importa as funções SQL
from pyspark.sql import functions as F
from pyspark.sql.functions import *

In [ ]:
#Quantidade de linhas no dataframe
linhas = df_spark.count()
print(linhas)

30746


In [ ]:
#Quantidade de colunas no dataframe
colunas = len(df_spark.columns)
print(colunas)

18


In [ ]:
#Selecionando apenas uma coluna
df_spark.select(df_spark.neighbourhood).show()

+---------------+
|  neighbourhood|
+---------------+
|     Copacabana|
|        Ipanema|
|        Ipanema|
|           Leme|
|     Copacabana|
|     Copacabana|
|     Copacabana|
|        Ipanema|
|        Ipanema|
|     Copacabana|
|     Copacabana|
|Barra da Tijuca|
|     Copacabana|
|  Vila da Penha|
|     Copacabana|
|     Copacabana|
|     Copacabana|
|       Flamengo|
|     Copacabana|
|            Joá|
+---------------+
only showing top 20 rows



In [ ]:
#Selecionando valores distintos
dropDisDF = df_spark.dropDuplicates(["neighbourhood"]).select("neighbourhood")
dropDisDF.show(truncate=False)

+-------------------+
|neighbourhood      |
+-------------------+
|-22.96015          |
|Cocotá             |
|-22.97178          |
|Gávea              |
|Tomás Coelho       |
|Ipanema            |
|-22.98487          |
|-23.012897663648584|
|-22.85223724479751 |
|-22.97122          |
|-22.98221          |
|-22.961974403551796|
|Realengo           |
|-22.99748          |
|-22.9775469        |
|Gardênia Azul      |
|-22.97505          |
|Rocha              |
|Bento Ribeiro      |
|Vicente de Carvalho|
+-------------------+
only showing top 20 rows



In [ ]:
#Realizando uma limpeza no dataframe
df_spark_clean = df_spark.filter(~F.col('neighbourhood').contains('2'))
dropDisDF_clean = df_spark_clean.dropDuplicates(["neighbourhood"]).select("neighbourhood")
dropDisDF_clean.show(truncate=False)

+-------------------+
|neighbourhood      |
+-------------------+
|Cocotá             |
|Gávea              |
|Tomás Coelho       |
|Ipanema            |
|Realengo           |
|Gardênia Azul      |
|Rocha              |
|Bento Ribeiro      |
|Vicente de Carvalho|
|Manguinhos         |
|Leme               |
|Jacarepaguá        |
|Botafogo           |
|Campo Grande       |
|Rocinha            |
|Grumari            |
|Padre Miguel       |
|Flamengo           |
|Santo Cristo       |
|Engenho da Rainha  |
+-------------------+
only showing top 20 rows



In [ ]:
#Verificando o total de reviews por bairro do Rio de Janeiro
df_spark_clean.groupBy('neighbourhood').sum('number_of_reviews').orderBy('neighbourhood').show(100)

+--------------------+----------------------+
|       neighbourhood|sum(number_of_reviews)|
+--------------------+----------------------+
|            Abolição|                     0|
|               Acari|                     0|
|   Alto da Boa Vista|                   558|
|            Anchieta|                     0|
|             Andaraí|                   256|
|                Anil|                   158|
|           Bancários|                    62|
|               Bangu|                    78|
|     Barra da Tijuca|                 48023|
|  Barra de Guaratiba|                  1524|
|        Barros Filho|                     1|
|             Benfica|                    49|
|       Bento Ribeiro|                   165|
|          Bonsucesso|                    29|
|            Botafogo|                 23499|
|        Brás de Pina|                    89|
|            Cachambi|                   123|
|              Cacuia|                   179|
|                Caju|            

In [ ]:
#Verificando o total de reviews por bairro do Rio de Janeiro
df_spark_clean.groupBy('neighbourhood').sum('number_of_reviews').orderBy('sum(number_of_reviews)').show(100)

+--------------------+----------------------+
|       neighbourhood|sum(number_of_reviews)|
+--------------------+----------------------+
|          Manguinhos|                     0|
|            Abolição|                     0|
|            Anchieta|                     0|
|         Vila Kosmos|                     0|
|          Santíssimo|                     0|
|               Acari|                     0|
|          Água Santa|                     0|
|                Caju|                     0|
|Ricardo de Albuqu...|                     0|
|          Cavalcanti|                     0|
|   Engenho da Rainha|                     1|
|              Pavuna|                     1|
|     Engenheiro Leal|                     1|
|           Cascadura|                     1|
|        Barros Filho|                     1|
|  Complexo do Alemão|                     1|
|            Cordovil|                     1|
|              Jacaré|                     1|
|            Gericinó|            

In [ ]:
df_spark_clean.groupBy('neighbourhood').sum('number_of_reviews').orderBy(col('sum(number_of_reviews)').desc()).show(100)

+--------------------+----------------------+
|       neighbourhood|sum(number_of_reviews)|
+--------------------+----------------------+
|          Copacabana|                215700|
|             Ipanema|                 80063|
|     Barra da Tijuca|                 48023|
|              Leblon|                 33750|
|            Botafogo|                 23499|
|              Centro|                 20007|
|        Santa Teresa|                 19064|
|Recreio dos Bande...|                 17189|
|         Jacarepaguá|                 15272|
|                Leme|                 10566|
|            Flamengo|                  9596|
|             Camorim|                  4906|
|              Glória|                  4866|
|         Laranjeiras|                  4556|
|              Tijuca|                  4280|
|             Vidigal|                  3493|
|              Catete|                  3426|
|               Lagoa|                  3240|
|                Urca|            